# Analyzing unstructured text in product review data

It's common for companies to have useful data hidden in large volumes of text:

- online reviews
- social media posts and tweets
- interactions with customers, such as emails and call center transcripts

For example, when shopping it can be challenging to decide between products with the same star rating. When this happens, shoppers often sift through the raw text of reviews to understand the strengths and weaknesses of each option.

<img src="ItemC.png">
<img src="ItemD.png">

In this notebook we seek to automate the task of determining product strengths and weaknesses from review text.

1. splitting Amazon review text into sentences and applying a sentiment analysis model
2. tagging documents that mention aspects of interest
3. extract adjectives from raw text, and compare their use in positive and negative reviews
4. summarizing the use of adjectives for tagged documents

GraphLab Create includes feature engineering objects that leverage spaCy, a high performance NLP package. Here we use it for extracting parts of speech and parsing reviews into sentences.

In [1]:
import graphlab as gl

In [2]:
from graphlab.toolkits.text_analytics import trim_rare_words, split_by_sentence, extract_part_of_speech, stopwords, PartOfSpeech

def nlp_pipeline(reviews, title, aspects):

    print(title)
    
    print('1. Get reviews for this product')
    reviews = reviews.filter_by(title, 'name')

    print('2. Splitting reviews into sentences')
    reviews['sentences'] = split_by_sentence(reviews['review'])
    sentences = reviews.stack('sentences', 'sentence').dropna()

    print('3. Tagging relevant reviews')
    tags = gl.SFrame({'tag': aspects})
    tagger_model = gl.data_matching.autotagger.create(tags, verbose=False)
    tagged = tagger_model.tag(sentences, query_name='sentence', similarity_threshold=.3, verbose=False)\
                         .join(sentences, on='sentence')

    print('4. Extracting adjectives')
    tagged['cleaned']    = trim_rare_words(tagged['sentence'], stopwords=list(stopwords()))
    tagged['adjectives'] = extract_part_of_speech(tagged['cleaned'], [PartOfSpeech.ADJ])

    print('5. Predicting sentence-level sentiment')
    model = gl.sentiment_analysis.create(tagged, features=['review'])
    tagged['sentiment']  = model.predict(tagged)
    return tagged

In [3]:
reviews = gl.SFrame('amazon_baby.gl')

2016-04-14 09:42:44,477 [INFO] graphlab.cython.cy_server, 176: GraphLab Create v1.9 started. Logging: /tmp/graphlab_server_1460652163.log


This commercial license of GraphLab Create is assigned to engr@turi.com.


In [4]:
reviews

review,rating,name
This book is amazing! Ibought it and read it to ...,5.0,Stop Pacifier Suckingwithout tears with ...
disappointed that thebook and puppet were so ...,2.0,Stop Pacifier Suckingwithout tears with ...
I have this nook and Ilove it. But this ...,5.0,A Tale of Baby's Dayswith Peter Rabbit ...
Perfect for new parents.We were able to keep ...,5.0,"Baby Tracker&reg; - DailyChildcare Journal, ..."
We have used this productsince our daughter was ...,5.0,"Baby Tracker&reg; - DailyChildcare Journal, ..."
Every new mom needs thisjournal. The 24 hour ...,5.0,"Baby Tracker&reg; - DailyChildcare Journal, ..."
"This is great for basics,but I wish the space to ...",4.0,"Baby Tracker&reg; - DailyChildcare Journal, ..."
I love this journal andour nanny uses it ...,4.0,"Baby Tracker&reg; - DailyChildcare Journal, ..."
My 3 month old son spendhalf of his days with my ...,5.0,"Baby Tracker&reg; - DailyChildcare Journal, ..."
"I wanted to love this,but it was pretty ...",3.0,"Baby Tracker&reg; - DailyChildcare Journal, ..."


In [5]:
from helper_util import *

## Focus on chosen aspects about baby monitors

In [6]:
aspects = ['audio', 'price', 'signal', 'range', 'battery life']

In [7]:
reviews = search(reviews, 'monitor')

Tokenizing...

TF-IDF transform...

Creating inverted index...

Creating query expansion model...

Saving data for querying...

In [8]:
reviews

review,rating,name
This baby monitor hasbeen working for us for ...,4.0,Baby Monitor - DirectLink Privacy Monitor ...
My sister recommended weget this monitor because ...,5.0,Graco ultraclear babymonitor ...
These monitors areabsolutly wonderful. ...,5.0,Graco ultraclear babymonitor ...
I have been using thismonitor for 10 months ...,5.0,Graco ultraclear babymonitor ...
After trying andreturning 3 different ...,5.0,Graco ultraclear babymonitor ...
This monitor has beengreat! It's always very ...,5.0,Graco ultraclear babymonitor ...
I have used this monitorfor three years with my ...,3.0,Graco ultraclear babymonitor ...
"Amazing monitor, I canhear every little sound ...",5.0,Graco ultraclear babymonitor ...
We were giving thismonitor as a gift since ...,2.0,Graco ultraclear babymonitor ...
1 is too many stars forthe product! ...,1.0,Fisher-price Super-sensitive Nursey Monitor ...


## Process reviews for the most common product

In [9]:
item_a = 'Infant Optics DXR-5 2.4 GHz Digital Video Baby Monitor with Night Vision'
reviews_a = nlp_pipeline(reviews, item_a, aspects)

Infant Optics DXR-5 2.4 GHz Digital Video Baby Monitor with Night Vision
1. Get reviews for this product
2. Splitting reviews into sentences
3. Tagging relevant reviews
4. Extracting adjectives
5. Predicting sentence-level sentiment


In [10]:
reviews_a

sentence_id,sentence,tag,score,review,rating
2,It killed our wifi signalthen lost it's pairing ...,signal,1.0,It killed our wifi signalthen lost it's pairing ...,1.0
4,Both audio and videomonitor supersedes the ...,audio,0.5,I love this videomonitor. Both audio and ...,5.0
4,Both audio and videomonitor supersedes the ...,price,0.5,I love this videomonitor. Both audio and ...,5.0
5,The VOX poewr savinghelps prevent quick ...,battery life,0.454545454545,I love this videomonitor. Both audio and ...,5.0
11,Definitely worth theprice. ...,price,0.666666666667,This is such a greatcamera. It doesn't pivot ...,5.0
17,I purchased it based onreviews and the appea ...,price,0.666666666667,I had bought this monitorback in july when my ...,1.0
18,First and foremost thebattery life SUCKS. ...,battery life,1.0,I had bought this monitorback in july when my ...,1.0
35,It's good and inexpensiveenough that we've dec ...,range,0.333333333333,I tried this after havingthe Lenox monitor. The ...,4.0
36,It's been hard to find avideo system that does ...,price,0.666666666667,I tried this after havingthe Lenox monitor. The ...,4.0
40,This is a great cameramonitor for the price. ...,price,0.666666666667,This is a great cameramonitor for the price. ...,4.0


## Comparing to another product

In [11]:
dropdown = get_dropdown(reviews)
display(dropdown)

In [12]:
item_b = dropdown.value
reviews_b = nlp_pipeline(reviews, item_b, aspects)
counts, sentiment, adjectives = get_comparisons(reviews_a, reviews_b, item_a, item_b, aspects)

VTech Communications Safe &amp; Sound Digital Audio Monitor
1. Get reviews for this product
2. Splitting reviews into sentences
3. Tagging relevant reviews
4. Extracting adjectives
5. Predicting sentence-level sentiment


Comparing the number of sentences that mention each aspect

In [13]:
counts

tag,Infant Optics DXR-5 2.4GHz Digital Video Baby ...,VTech Communications Safe&amp; Sound Digital A ...
signal,107,15
battery life,180,93
range,144,68
audio,105,27
price,251,69


Comparing the sentence-level sentiment for each aspect of each product

In [14]:
sentiment

tag,Infant Optics DXR-5 2.4GHz Digital Video Baby ...,VTech Communications Safe&amp; Sound Digital A ...
signal,0.674221231596,0.695281129091
battery life,0.761379749571,0.831840808502
range,0.862246778561,0.840649624394
audio,0.826972950724,0.92381971402
price,0.885832746604,0.914282565406


Comparing the use of adjectives for each aspect

In [ ]:
adjectives

## Investigating good and bad sentences

In [ ]:
good, bad = get_extreme_sentences(reviews_a)

Print good sentences for the first item, where adjectives and aspects are highlighted.

In [ ]:
print_sentences(good['highlighted'])

Print bad sentences for the first item, where adjectives and aspects are highlighted.

In [ ]:
print_sentences(bad['highlighted'])

# Deployment

In [17]:
service = gl.deploy.predictive_service.load("s3://gl-demo-usw2/predictive_service/demolab/ps-1.8.5")

2016-04-14 10:33:45,329 [WARNING] graphlab.deploy.predictive_service, 384: Overwriting existing Predictive Service "demolab-ps-one-eight-five" in local session.


In [18]:
service.get_predictive_objects_status()

ConnectionError: HTTPSConnectionPool(host='demolab-ps-one-eight-five-1964664870.us-west-2.elb.amazonaws.com', port=443): Max retries exceeded with url: /manage/status (Caused by NewConnectionError('<requests.packages.urllib3.connection.VerifiedHTTPSConnection object at 0x7ff8ec0b8210>: Failed to establish a new connection: [Errno 111] Connection refused',))

In [ ]:
def word_count(text):
    sa = gl.SArray([text])
    sa = gl.text_analytics.count_words(sa)
    return sa[0]

In [ ]:
service.update('chris_bow', word_count)

In [ ]:
service.apply_changes()

In [ ]:
service.query('chris_bow', text=["It's a beautiful day in the neighborhood. Beautiful day for a neighbor."])